# Year-Normalized Citation Target

**CRITICAL INSIGHT:** Current target uses fixed threshold (26 citations) across all years.

**Problem:**
- 2015 papers had ~7 years to accumulate citations (by 2022)
- 2020 papers had ~2 years to accumulate citations
- A 2020 paper with 26 citations is MORE impressive than a 2015 paper with 26 citations
- Model may be learning "older = more citations" instead of "quality = more citations"

**Solution:** Use year-stratified thresholds (top 25% WITHIN each year)

**Expected outcome:** F1 should improve if temporal bias was limiting performance

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

## 1. Load Data and Analyze Citation Distribution by Year

In [ ]:
# Load cleaned dataset
df = pd.read_pickle('../data/processed/cleaned_data.pkl')

# Load existing features
X_all = pd.read_pickle('../data/features/X_all.pkl')
metadata = pd.read_pickle('../data/features/metadata.pkl')

print(f"Dataset: {df.shape}")
print(f"Features: {X_all.shape}")
print(f"\nYear range: {df['Year'].min()} - {df['Year'].max()}")

In [ ]:
# Analyze citation distribution by year
print("=" * 60)
print("CITATION STATISTICS BY YEAR")
print("=" * 60)

year_stats = df.groupby('Year')['Citations'].agg([
    'count', 'mean', 'median', 'std',
    ('25th_pct', lambda x: x.quantile(0.25)),
    ('75th_pct', lambda x: x.quantile(0.75)),
    ('90th_pct', lambda x: x.quantile(0.90))
]).round(2)

print(year_stats)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Mean citations by year
year_stats['mean'].plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('Mean Citations by Year')
axes[0].set_xlabel('Year')
axes[0].set_ylabel('Mean Citations')
axes[0].grid(axis='y', alpha=0.3)

# 75th percentile by year (current threshold is global 75th)
year_stats['75th_pct'].plot(kind='bar', ax=axes[1], color='coral')
axes[1].axhline(y=26, color='red', linestyle='--', label='Global 75th percentile (26)')
axes[1].set_title('75th Percentile Citations by Year')
axes[1].set_xlabel('Year')
axes[1].set_ylabel('Citations (75th percentile)')
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n⚠️  OBSERVATION:")
print(f"   Older years have higher mean/median citations (more time to accumulate)")
print(f"   Using a fixed threshold (26) biases toward older papers")

## 2. Create Year-Normalized Target Variable

In [ ]:
print("=" * 60)
print("CREATING YEAR-NORMALIZED TARGET")
print("=" * 60)

# Method: For each year, mark top 25% as high-impact
def create_year_normalized_target(df, percentile=0.75):
    """
    Create target where high-impact = top 25% WITHIN each year.
    This accounts for different citation accumulation times.
    """
    df = df.copy()
    df['high_impact_normalized'] = 0
    
    year_thresholds = {}
    
    for year in df['Year'].unique():
        year_mask = df['Year'] == year
        threshold = df.loc[year_mask, 'Citations'].quantile(percentile)
        df.loc[year_mask, 'high_impact_normalized'] = (
            df.loc[year_mask, 'Citations'] >= threshold
        ).astype(int)
        year_thresholds[year] = threshold
    
    return df, year_thresholds

df_normalized, year_thresholds = create_year_normalized_target(df, percentile=0.75)

print("\nYear-specific thresholds (75th percentile):")
for year, threshold in sorted(year_thresholds.items()):
    count = (df_normalized['Year'] == year).sum()
    high_impact = ((df_normalized['Year'] == year) & (df_normalized['high_impact_normalized'] == 1)).sum()
    print(f"  {year}: {threshold:6.1f} citations ({high_impact}/{count} papers = {high_impact/count*100:.1f}%)")

print(f"\nGlobal distribution (normalized):")
print(df_normalized['high_impact_normalized'].value_counts())
print(f"High-impact: {df_normalized['high_impact_normalized'].mean()*100:.1f}%")

## 3. Compare Old vs New Target

In [ ]:
# Load old target for comparison
y_old = pd.read_pickle('../data/features/y_classification.pkl')

# Align indices
y_new = df_normalized.loc[X_all.index, 'high_impact_normalized']

print("=" * 60)
print("OLD TARGET vs NEW TARGET COMPARISON")
print("=" * 60)

# Compare
comparison = pd.DataFrame({
    'Old (Fixed 26)': y_old,
    'New (Year-Normalized)': y_new
})

print("\nCross-tabulation:")
print(pd.crosstab(comparison['Old (Fixed 26)'], comparison['New (Year-Normalized)'], 
                   rownames=['Old'], colnames=['New'], margins=True))

# How many labels changed?
changed = (comparison['Old (Fixed 26)'] != comparison['New (Year-Normalized)']).sum()
pct_changed = changed / len(comparison) * 100

print(f"\nLabels changed: {changed} / {len(comparison)} ({pct_changed:.1f}%)")
print(f"\nThis means year normalization affects {pct_changed:.1f}% of labels.")
print(f"If model was learning temporal bias, this should improve F1!")

## 4. Create Temporal Train/Test Split with New Target

In [ ]:
# Same temporal split as before (2015-2017 train, 2018-2020 test)
train_mask = metadata['Year'].isin([2015, 2016, 2017])
test_mask = metadata['Year'].isin([2018, 2019, 2020])

X_train = X_all[train_mask]
X_test = X_all[test_mask]
y_train_new = y_new[train_mask]
y_test_new = y_new[test_mask]

print("=" * 60)
print("TEMPORAL SPLIT WITH NEW TARGET")
print("=" * 60)
print(f"\nTrain (2015-2017): {X_train.shape}")
print(f"  High-impact: {y_train_new.sum()} / {len(y_train_new)} ({y_train_new.mean()*100:.1f}%)")
print(f"\nTest (2018-2020): {X_test.shape}")
print(f"  High-impact: {y_test_new.sum()} / {len(y_test_new)} ({y_test_new.mean()*100:.1f}%)")

## 5. Train Baseline Model with OLD Target (for comparison)

In [ ]:
# Load old targets
y_train_old = pd.read_pickle('../data/features/y_train_cls_temporal.pkl')
y_test_old = pd.read_pickle('../data/features/y_test_cls_temporal.pkl')

print("=" * 60)
print("BASELINE - OLD TARGET (Fixed 26 citations)")
print("=" * 60)

model_old = LogisticRegression(
    max_iter=1000,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

model_old.fit(X_train, y_train_old)
y_pred_proba_old = model_old.predict_proba(X_test)[:, 1]
y_pred_old = (y_pred_proba_old >= 0.54).astype(int)

old_results = {
    'Accuracy': accuracy_score(y_test_old, y_pred_old),
    'Precision': precision_score(y_test_old, y_pred_old),
    'Recall': recall_score(y_test_old, y_pred_old),
    'F1': f1_score(y_test_old, y_pred_old),
    'ROC-AUC': roc_auc_score(y_test_old, y_pred_proba_old)
}

print("\nResults with OLD target:")
for metric, value in old_results.items():
    print(f"  {metric:12s}: {value*100:.2f}%")

print("\nConfusion Matrix (OLD):")
print(confusion_matrix(y_test_old, y_pred_old))

## 6. Train Model with NEW Year-Normalized Target

In [ ]:
print("\n" + "=" * 60)
print("NEW MODEL - YEAR-NORMALIZED TARGET")
print("=" * 60)

model_new = LogisticRegression(
    max_iter=1000,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'
)

model_new.fit(X_train, y_train_new)
y_pred_proba_new = model_new.predict_proba(X_test)[:, 1]
y_pred_new = (y_pred_proba_new >= 0.54).astype(int)

new_results = {
    'Accuracy': accuracy_score(y_test_new, y_pred_new),
    'Precision': precision_score(y_test_new, y_pred_new),
    'Recall': recall_score(y_test_new, y_pred_new),
    'F1': f1_score(y_test_new, y_pred_new),
    'ROC-AUC': roc_auc_score(y_test_new, y_pred_proba_new)
}

print("\nResults with NEW target:")
for metric, value in new_results.items():
    print(f"  {metric:12s}: {value*100:.2f}%")

print("\nConfusion Matrix (NEW):")
print(confusion_matrix(y_test_new, y_pred_new))

## 7. Compare Results

In [ ]:
print("\n" + "=" * 80)
print("COMPARISON: OLD TARGET vs YEAR-NORMALIZED TARGET")
print("=" * 80)

comparison_df = pd.DataFrame({
    'Old Target (Fixed 26)': old_results,
    'New Target (Year-Normalized)': new_results,
    'Change': {k: new_results[k] - old_results[k] for k in old_results.keys()}
}).T

# Format as percentages
for col in comparison_df.columns:
    comparison_df[col] = comparison_df[col].apply(lambda x: f"{x*100:+.2f}%" if isinstance(x, float) else x)

print("\n", comparison_df)

# Determine if improvement
f1_improvement = new_results['F1'] - old_results['F1']

print("\n" + "=" * 80)
if f1_improvement > 0.01:  # More than 1 percentage point
    print(f"✅ SIGNIFICANT IMPROVEMENT!")
    print(f"   F1 improved from {old_results['F1']*100:.2f}% → {new_results['F1']*100:.2f}%")
    print(f"   Gain: +{f1_improvement*100:.2f} points")
    print(f"\n   Year normalization WAS the issue limiting F1!")
    print(f"   The model is actually better than we thought.")
elif f1_improvement > 0:
    print(f"⚠️  SLIGHT IMPROVEMENT (+{f1_improvement*100:.2f} F1)")
    print(f"   Year normalization helps marginally")
    print(f"   F1: {old_results['F1']*100:.2f}% → {new_results['F1']*100:.2f}%")
else:
    print(f"❌ NO IMPROVEMENT ({f1_improvement*100:+.2f} F1)")
    print(f"   Year normalization didn't help")
    print(f"   Temporal bias wasn't the limiting factor")
    print(f"   Original 62.54% F1 remains best")

print("=" * 80)

## 8. Optimize Threshold for New Target (if improved)

In [ ]:
if f1_improvement > 0:
    print("\n" + "=" * 60)
    print("THRESHOLD OPTIMIZATION FOR NEW TARGET")
    print("=" * 60)
    
    # Test thresholds
    thresholds = np.arange(0.45, 0.66, 0.01)
    threshold_results = []
    
    for thresh in thresholds:
        y_pred = (y_pred_proba_new >= thresh).astype(int)
        f1 = f1_score(y_test_new, y_pred)
        prec = precision_score(y_test_new, y_pred)
        rec = recall_score(y_test_new, y_pred)
        threshold_results.append({
            'threshold': thresh,
            'f1': f1,
            'precision': prec,
            'recall': rec
        })
    
    best_thresh_row = max(threshold_results, key=lambda x: x['f1'])
    best_threshold = best_thresh_row['threshold']
    
    print(f"\nBest threshold: {best_threshold:.3f}")
    print(f"F1 at best threshold: {best_thresh_row['f1']*100:.2f}%")
    print(f"Precision: {best_thresh_row['precision']*100:.2f}%")
    print(f"Recall: {best_thresh_row['recall']*100:.2f}%")
    
    if best_thresh_row['f1'] > new_results['F1']:
        print(f"\n✅ Optimized threshold improves F1 by {(best_thresh_row['f1'] - new_results['F1'])*100:.2f} points!")
else:
    print("\nSkipping threshold optimization (no improvement from year normalization)")

## 9. Final Recommendation

In [ ]:
print("\n💡 FINAL RECOMMENDATION:")
print("-" * 80)

if f1_improvement > 0.01:
    print(f"✅ USE YEAR-NORMALIZED TARGET!")
    print(f"\n   The fixed 26-citation threshold was biased toward older papers.")
    print(f"   Year normalization corrects this and reveals true model performance.")
    print(f"\n   NEW BEST RESULT:")
    print(f"     F1: {new_results['F1']*100:.2f}%")
    print(f"     ROC-AUC: {new_results['ROC-AUC']*100:.2f}%")
    print(f"     Precision: {new_results['Precision']*100:.2f}%")
    print(f"     Recall: {new_results['Recall']*100:.2f}%")
    print(f"\n   This is your TRUE performance - use this for your thesis!")
    
    # Save new targets
    print(f"\n   Saving year-normalized targets...")
    output_dir = Path('../data/features')
    y_new.to_pickle(output_dir / 'y_classification_normalized.pkl')
    y_train_new.to_pickle(output_dir / 'y_train_cls_normalized.pkl')
    y_test_new.to_pickle(output_dir / 'y_test_cls_normalized.pkl')
    print(f"   ✓ Saved normalized targets")
    
else:
    print(f"⚠️  Year normalization didn't improve F1")
    print(f"\n   Temporal bias wasn't the limiting factor.")
    print(f"   The original approach (fixed threshold) remains valid.")
    print(f"\n   BEST RESULT: {old_results['F1']*100:.2f}% F1 (original)")
    print(f"\n   This confirms 62.54% F1 is the ceiling with current features.")

print("-" * 80)